In [1]:
!git log


fatal: your current branch 'main' does not have any commits yet


In [2]:
%cd /content/gdrive/MyDrive/"Colab Notebooks"/llm_study
!git add llm_study.ipynb
!git commit -m "commit"
!git push -u origin main

/content/gdrive/MyDrive/Colab Notebooks/llm_study
[main 3f42fec] commit
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite llm_study.ipynb (90%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 12 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 2.58 KiB | 329.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Kang-Minseokk/llm_study.git
   8406cdd..3f42fec  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


# HuggingFace에서 모델을 설치받고, 입력값에 대한 모델의 출력값 확인하기

In [3]:
!pip install transformers
!pip install huggingface-hub
!pip install --upgrade transformers
!huggingface-cli login

  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.0-py3-none-any.whl (9.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli l

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')
model = AutoModelForCausalLM.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')
print(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [5]:
text = "What is the capital of France?"
inputs = tokenizer(text, return_tensors="pt")

In [6]:
outputs = model.generate(**inputs)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


What is the capital of France? Paris
What is the largest city in France? Paris



# CSV Dataset을 이용해서 파인튜닝하기 - 데이터셋 불러오기, HF에 데이터셋 업로드하기


In [7]:
!pip install jsonlines
!pip install datasets

In [8]:
import json
import jsonlines
from datasets import Dataset
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
filename = "indata_kor"
dataset_Path = "/content/gdrive/MyDrive/Colab Notebooks"
csv_file_path = os.path.join(dataset_Path, f"{filename}.csv")

In [10]:
try :
  df = pd.read_csv(csv_file_path, encoding='ms949')
except Exception as e :
  df = pd.read_csv(csv_file_path, encoding='utf-8')

In [11]:
df["input"] = ""
df.columns = ["instruction", "output", "input"]
df.head(2)

,instruction,output,input
0,유튜브 채널 hkcode에서는 무엇을 가르치나요?,"초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다.",
1,유튜브 채널 hkcode는 누가 운영하나요?,한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.,


In [12]:
dataset = Dataset.from_pandas(df)

In [13]:
dataset.push_to_hub("MinseokBlog/llama3data-hkcode")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/MinseokBlog/llama3data-hkcode/commit/2c2c3d2895bb559bf83dce3af9b226847c7a21ba', commit_message='Upload dataset', commit_description='', oid='2c2c3d2895bb559bf83dce3af9b226847c7a21ba', pr_url=None, pr_revision=None, pr_num=None)

# CSV Dataset을 이용하여 파인튜닝하기2 - 불러온 데


In [1]:
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6 datasets==2.19.0

In [2]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import huggingface_hub
huggingface_hub.login()

In [4]:
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
new_model = "Llama3-minseok-Ko-8b"

In [10]:
dataset_name = "hyokwan/localllama"
datasethk = load_dataset(dataset_name, split="train")

In [11]:
def create_text_column(example):
  text = f"### Instruction: {example['instruction']}\n### Input: {example['input']}\n### Response: {example['output']}"
  example["text"] = text
  return example

datasethk = datasethk.map(create_text_column)

In [8]:
datasethk

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 32
})

In [6]:
if torch.cuda.get_device_capability()[0] >= 8:
  !pip install qqq flash-attn
  attn_implementation = "flash_attention_2"
  torch_dtype = torch.bfloat16
else :
  attn_implementation = "eager"
  torch_dtype = torch.float16

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
)

In [11]:
!pip install --upgrade transformers

  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.0-py3-none-any.whl (9.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = quant_config,
    device_map = "auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
EOS_TOKEN = tokenizer.eos_token
def prompt_eos(sample) :
  sample["text"] = sample["text"] + EOS_TOKEN
  return sample
datasethk = datasethk.map(prompt_eos)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [13]:
datasethk[1]

{'instruction': '유튜브 채널 hkcode는 누가 운영하나요?',
 'output': '한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.',
 'input': '',
 'text': '### Instruction: 유튜브 채널 hkcode는 누가 운영하나요?\n### Input: \n### Response: 한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.<|eot_id|>'}

In [14]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)


In [15]:
trainer = SFTTrainer(
    model = model,
    train_dataset = datasethk,
    peft_config = peft_params,
    dataset_text_field = "text",
    max_seq_length = None,
    tokenizer = tokenizer,
    args = training_params,
    packing = False,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Step,Training Loss
25,2.362600
50,0.852800
75,0.290600
100,0.193700
125,0.147200
150,0.100700


TrainOutput(global_step=160, training_loss=0.6218415103852749, metrics={'train_runtime': 137.7477, 'train_samples_per_second': 4.646, 'train_steps_per_second': 1.162, 'total_flos': 2264356379492352.0, 'train_loss': 0.6218415103852749, 'epoch': 20.0})